In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import mysql.connector
import import_ipynb
import re
import pandas as pd
from algoritma.MMR import *
from nltk.corpus import stopwords
import string
import csv
import time
import string
import nltk
import spacy
from nltk.stem import PorterStemmer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

importing Jupyter notebook from C:\Users\risma\Kuliah\MCA\Text-Summarization-News-Ariticles\algoritma\MMR.ipynb


In [2]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="db-berita-mca"  
) 

In [3]:
def tokenizeParagraftoKalimat(paragraf):
    tokenize = sent_tokenize(paragraf)
    return tokenize

In [4]:
def cleansingData(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    hasil = re.sub('[^A-Za-z0-9 ]+', '', words_wo_punct)
    hasil = hasil.translate(str.maketrans("","",string.punctuation))
    return hasil

In [5]:
def steamingData(sentence):
    porter = PorterStemmer()
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [6]:
def stopwordEN(text):
    hasilStopWord = []
    stopwordNLTK = nltk.corpus.stopwords.words('english')
    stopwordSpacy = list(STOP_WORDS)
    for word in text.split():
        if not word in stopwordNLTK:
            if not word in stopwordSpacy:
                hasilStopWord.append(word)
    return " ".join(hasilStopWord)

In [7]:
def hasilSummary(kalimatAsli, summarySet):
    hasil = []
    for sentence in summarySet:
        hasil.append(kalimatAsli [sentence].lstrip(' ')) 
    return " ".join(hasil)

In [8]:
def visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli):
    for sentence in cleansingTeks:
        if sentence in summarySet:
            print(colored(kalimatAsli[sentence].lstrip(' '),'red'))
        else:
            print(kalimatAsli[sentence].lstrip(' '))

In [9]:
def analysisSentiment(sentence):
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)
    score = ''
    labeling = ''
    
    if sentiment_dict['compound'] > 0:
        labeling = "Positive"
 
    elif sentiment_dict['compound'] < 0 :
        labeling = "Negative"
 
    elif sentiment_dict['compound'] == 0 :
        labeling = "Neutral"
        
    return labeling,sentiment_dict['compound']

In [10]:
def simpanData(mydb,title,content,subject,date,summary,labeling_content,score_content,labeling_summary,score_summary):
    mycursor = mydb.cursor()
    sql = "INSERT INTO tb_result_dataset (title,content,subject,date,summary,labeling_content,score_content,labeling_summary,score_summary) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)" 
    param = (title,content,subject,date,summary,labeling_summary,score_summary,labeling_content,score_content)
    mycursor.execute(sql, param)
    mydb.commit()

In [12]:
df = pd.read_csv("data/dataset.csv")
for iteration, data in enumerate(df['text']):
    title = df['title'][iteration]
    text = df['text'][iteration]
    subject = df['subject'][iteration]
    date = df['date'][iteration]
    
    tokenizeKalimat = (tokenizeParagraftoKalimat(data)) 
    
    cleansingTeks = []
    kalimatAsli = {}
    
    for text in tokenizeKalimat:
        steam = steamingData(cleansingData(text))
        dataFinal = stopwordEN(steam)
        cleansingTeks.append(dataFinal)
        kalimatAsli[dataFinal] = text
    
    cleansingTeks = [x for x in cleansingTeks if x]
    iteration = MMR()
    score = iteration.similarityScore(cleansingTeks)
    summarySet = iteration.calculateMMR(score,cleansingTeks)
    
    summary = hasilSummary(kalimatAsli,summarySet)

#     visualisasiSummaryOri(cleansingTeks,summarySet,kalimatAsli)
    
    sentimentContent,scoreContent = analysisSentiment(cleansingTeks)
    sentimentSummary,scoreSummary = analysisSentiment(summarySet)
    
    print(sentimentContent,scoreContent)
    print("-----------------------------------")
    print(sentimentSummary,scoreSummary)
    print("\n")
    
    simpanData(mydb,title,text,subject,date,summary,sentimentContent,scoreContent,sentimentSummary,scoreSummary)
        

Positive 0.9666
-----------------------------------
Positive 0.7506


Positive 0.9413
-----------------------------------
Positive 0.6124


Positive 0.8176
-----------------------------------
Negative -0.4588


Positive 0.9517
-----------------------------------
Positive 0.7351


Positive 0.9887
-----------------------------------
Positive 0.7845


Positive 0.8834
-----------------------------------
Negative -0.0772


Positive 0.9842
-----------------------------------
Positive 0.7579


Positive 0.4588
-----------------------------------
Neutral 0.0


Positive 0.7003
-----------------------------------
Neutral 0.0


Positive 0.3612
-----------------------------------
Positive 0.5859


Negative -0.7003
-----------------------------------
Negative -0.1027


Positive 0.9578
-----------------------------------
Positive 0.3612


Positive 0.4588
-----------------------------------
Positive 0.1531


Positive 0.7269
-----------------------------------
Negative -0.0516


Positive 0.8834
-------

Positive 0.25
-----------------------------------
Negative -0.743


Positive 0.5106
-----------------------------------
Positive 0.3818


Positive 0.9682
-----------------------------------
Positive 0.6908


Positive 0.9831
-----------------------------------
Positive 0.8658


Positive 0.4959
-----------------------------------
Positive 0.5719


Positive 0.906
-----------------------------------
Positive 0.7506


Negative -0.9169
-----------------------------------
Negative -0.7003


Positive 0.4939
-----------------------------------
Negative -0.1027


Positive 0.9805
-----------------------------------
Positive 0.6597


Negative -0.5994
-----------------------------------
Negative -0.5859


Positive 0.7096
-----------------------------------
Positive 0.7096


Positive 0.7579
-----------------------------------
Positive 0.7579


Positive 0.9545
-----------------------------------
Positive 0.9153


Positive 0.4404
-----------------------------------
Positive 0.4404


Positive 0.9186
--

Positive 0.9186
-----------------------------------
Positive 0.5994


Positive 0.9413
-----------------------------------
Positive 0.7579


Positive 0.9545
-----------------------------------
Positive 0.7783


Negative -0.7003
-----------------------------------
Negative -0.5423


Negative -0.5423
-----------------------------------
Positive 0.25


Negative -0.9846
-----------------------------------
Negative -0.8481


Positive 0.0772
-----------------------------------
Positive 0.0772


Positive 0.9468
-----------------------------------
Positive 0.34


Positive 0.9937
-----------------------------------
Positive 0.9081


Positive 0.6597
-----------------------------------
Negative -0.0258


Negative -0.0258
-----------------------------------
Positive 0.8658


Negative -0.802
-----------------------------------
Negative -0.8442


Positive 0.6597
-----------------------------------
Negative -0.0258


Positive 0.9829
-----------------------------------
Positive 0.8176


Negative -0.875

Negative -0.7964
-----------------------------------
Negative -0.765


Positive 0.9163
-----------------------------------
Negative -0.5574


Positive 0.5574
-----------------------------------
Neutral 0.0


Positive 0.9231
-----------------------------------
Positive 0.7717


Positive 0.9477
-----------------------------------
Positive 0.6249


Positive 0.0516
-----------------------------------
Negative -0.296


Positive 0.765
-----------------------------------
Negative -0.2732


Negative -0.1779
-----------------------------------
Negative -0.25


Negative -0.4404
-----------------------------------
Neutral 0.0


Positive 0.7003
-----------------------------------
Positive 0.34


Positive 0.9094
-----------------------------------
Positive 0.2911


Positive 0.63
-----------------------------------
Negative -0.1531


Positive 0.9136
-----------------------------------
Positive 0.4019


Negative -0.765
-----------------------------------
Negative -0.296


Negative -0.34
-------------

Negative -0.9524
-----------------------------------
Negative -0.4939


Positive 0.3182
-----------------------------------
Positive 0.3182


Positive 0.872
-----------------------------------
Negative -0.4939


Negative -0.34
-----------------------------------
Positive 0.2023


Negative -0.7906
-----------------------------------
Negative -0.4215


Positive 0.4404
-----------------------------------
Neutral 0.0


Positive 0.6705
-----------------------------------
Positive 0.4404


Negative -0.8591
-----------------------------------
Negative -0.4767


Positive 0.128
-----------------------------------
Neutral 0.0


Positive 0.5719
-----------------------------------
Positive 0.3612


Negative -0.9571
-----------------------------------
Negative -0.2023


Negative -0.9042
-----------------------------------
Negative -0.7845


Positive 0.7003
-----------------------------------
Positive 0.1779


Negative -0.8442
-----------------------------------
Negative -0.8885


Negative -0.8381
-